In [1]:
import pandas as pd
import time

In [2]:
df_train = pd.read_csv("data/technical/sales_train.csv", parse_dates=['date'])

In [3]:
all_date_stores_open = df_train[['date_block_num']].copy().drop_duplicates()

In [4]:
all_date_stores_open.describe()

,date_block_num
count,34.000000
mean,16.500000
std,9.958246
min,0.000000
25%,8.250000
50%,16.500000
75%,24.750000
max,33.000000


In [5]:
all_date_stores_open.head()

,date_block_num
0,0
115690,1
224303,2
345650,3
439759,4


In [6]:
all_ids_for_submission = pd.read_csv("data/technical/sample_submission.csv")

In [7]:
all_ids_for_submission = all_ids_for_submission['ID']

In [8]:
timestamp = time.time()

matrix = []

# create a matrix containing all the time series for each and every item sold that
# needs to be predicted.
# start off by finding all records of sold items and on which day.
# then merge on a list containing all the dates that have to be taken into account
# during the period of the time series
# then reset and sort the index.
# finally, add the tuple of the item_id and the dataframe containing all records into
# the final matrix
for item in all_ids_for_submission:
    item_transactions = df_train.loc[df_train['item_id'] == item]
    item_transactions = item_transactions[['date_block_num', 'item_cnt_day']].copy()
    item_transactions = item_transactions.groupby(['date_block_num'])['item_cnt_day'].sum()

    temp_date_df = all_date_stores_open
    temp_df = pd.merge(item_transactions, temp_date_df, how = 'outer', on=['date_block_num'])
    temp_df.fillna(0, inplace=True)
    temp_df.set_index('date_block_num', inplace=True, drop=True)
    temp_df.sort_index(inplace=True)
    matrix.append((item, temp_df))

print(time.time() - timestamp)

601.1223080158234


In [9]:
import pmdarima.arima as pm

# https://alkaline-ml.com/pmdarima/modules/classes.html
# pm.ndiffs(matrix[4656][1], test="adf")


In [10]:
# http://alkaline-ml.com/pmdarima/modules/generated/pmdarima.arima.AutoARIMA.html#pmdarima.arima.AutoARIMA


# out.summary()

In [11]:
timestamp = time.time()


df_submissions = pd.DataFrame(columns=['ID', 'item_cnt_month'])


for ts_tuple in matrix:

    ts = ts_tuple[1]

    number_of_zero_entries = ts.value_counts().get(0)

    try:
        if number_of_zero_entries != 34:
            model = pm.auto_arima(ts)
            prediction = model.predict(n_periods=1)[0]
        else:
            prediction = 0
    except ValueError:
        print(f"Error at matrix number {ts_tuple[0]}")

    index = ts_tuple[0]

    # append DF
    data = {
        'ID': [index],
        'item_cnt_month': [prediction]
    }
    df_data = pd.DataFrame(data)
    df_submissions = df_submissions.append(df_data, ignore_index=True)

print(time.time() - timestamp)

/Users/ced/opt/miniconda3/envs/rp/lib/python3.8/site-packages/pmdarima/arima/_auto_solvers.py:522: ModelFitWarning: Error fitting  ARIMA(5,0,0)(0,0,0)[0] intercept (if you do not want to see these warnings, run with error_action="ignore").
Traceback:
Traceback (most recent call last):
  File "/Users/ced/opt/miniconda3/envs/rp/lib/python3.8/site-packages/pmdarima/arima/_auto_solvers.py", line 506, in _fit_candidate_model
    fit.fit(y, X=X, **fit_params)
  File "/Users/ced/opt/miniconda3/envs/rp/lib/python3.8/site-packages/pmdarima/arima/arima.py", line 482, in fit
    self._fit(y, X, **fit_args)
  File "/Users/ced/opt/miniconda3/envs/rp/lib/python3.8/site-packages/pmdarima/arima/arima.py", line 403, in _fit
    fit, self.arima_res_ = _fit_wrapper()
  File "/Users/ced/opt/miniconda3/envs/rp/lib/python3.8/site-packages/pmdarima/arima/arima.py", line 393, in _fit_wrapper
    return arima, arima.fit(start_params=start_params,
  File "/Users/ced/opt/miniconda3/envs/rp/lib/python3.8/site-pac

Error at matrix number 9836


/Users/ced/opt/miniconda3/envs/rp/lib/python3.8/site-packages/pmdarima/arima/_auto_solvers.py:522: ModelFitWarning: Error fitting  ARIMA(4,0,0)(0,0,0)[0]           (if you do not want to see these warnings, run with error_action="ignore").
Traceback:
Traceback (most recent call last):
  File "/Users/ced/opt/miniconda3/envs/rp/lib/python3.8/site-packages/pmdarima/arima/_auto_solvers.py", line 506, in _fit_candidate_model
    fit.fit(y, X=X, **fit_params)
  File "/Users/ced/opt/miniconda3/envs/rp/lib/python3.8/site-packages/pmdarima/arima/arima.py", line 482, in fit
    self._fit(y, X, **fit_args)
  File "/Users/ced/opt/miniconda3/envs/rp/lib/python3.8/site-packages/pmdarima/arima/arima.py", line 403, in _fit
    fit, self.arima_res_ = _fit_wrapper()
  File "/Users/ced/opt/miniconda3/envs/rp/lib/python3.8/site-packages/pmdarima/arima/arima.py", line 393, in _fit_wrapper
    return arima, arima.fit(start_params=start_params,
  File "/Users/ced/opt/miniconda3/envs/rp/lib/python3.8/site-pac

7187.282856941223


In [12]:
df_submissions.to_csv('arima_submission.csv', index=False)